In [1]:
## Two-Class EEG Classification using LSTM-based RNN
## Here, we develop a baseline model for using an RNN to perform time-series prediction for one of two motor
## imagery (MI) tasks. The data from the file CLA_SubjectJ-170508-3St-LRHand-Inter.mat is processed and we
## only keep the data points that pertain to the left and right hand MI tasks. Each of the MI task data consists
## of 170 time steps, with each time step consisting of 22 inputs. In other words, we pass the current sample for
## of the 22 channels for each time step. At the end of the 170 time step, we send the output of the LSTM layers
## to a Dense layer with two outputs. To ease the classification at the outer layer, we have converted the class
## into a one-hot encoding of zeros and ones, and use the sigmoid unit at the output to match the output range.

## Preliminary results:
## Performance on training set is 100% classification rate after around 30 to 40 epochs.
## Performance on test set is around 60% - 68%.

## Potential problems:
## The total amount of data is relatively low. Can it be possible to combine multiple subject's L/R Hand MI data
## into one unified data set?
## Can also possible increase the size of the data set by doing some data augmentations.
## For example, injecting some noise into the existing EEG data to create more test data?

## Followup/TODO:
## Use pre-implemented common spatial patterns for feature extraction: avialable here: https://github.com/spolsley/common-spatial-patterns
## Try using a relatively simple non-linear energy operator as a pre-emphasis step on the data to see if it helps generalize
## the model.

In [2]:
## This file is used to split data into series of arrays and their corresponding MI task.
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import os

dataDir = "dataset/CLA/" #replace with folder the CLA files are in
dataList = []
markersArrayList=[]
for file in os.listdir( dataDir ) : #loads all CLA mat files
    temp=sio.loadmat(dataDir+file)
    tempO=temp['o'][0][0]
    tempData=tempO['data']
    tempData=np.transpose(tempData)
    tempData=tempData[0:21,:] #ignore 22nd channel
    dataList.append(tempData)
    tempMarkers=tempO['marker']
    tempMarkersArray = []
    for tempMarker in tempMarkers:
        tempMarkersArray.append(tempMarker[0])
    tempMarkersArray = np.asarray(tempMarkersArray)
    markersArrayList.append(tempMarkersArray)
data=np.concatenate(dataList, axis=1)
markersArray=np.concatenate(markersArrayList)

In [3]:
if 0: #for 1 loading just 1 .mat file
    ## This file is used to split data into series of arrays and their corresponding MI task.
    import scipy.io as sio
    import matplotlib.pyplot as plt
    import numpy as np
    file = sio.loadmat('../mat_files/CLA-SubjectJ-170508-3St-LRHand-Inter.mat') #replace with .mat file name
    header=file['__header__']
    version=file['__version__']
    glob=file['__globals__']
    ans=file['ans']


    x=file['x']
    o=file['o'][0][0]
    data=o['data']
    data = np.transpose(data)
    print(data)
    nS=o['nS'][0][0]
    #values of structure seem to be 2D numpy arrays, if originally a scalar in Matlab.
    #use [0][0] to get scalar.
    print("Number of samples: {numSamples}".format(numSamples=nS))
    test=o['id'][0] #id value became a 1D array of size 1 for some reason. use [0] to get value
    print("Dataset ID: {id}".format(id=test))
    chnames=o['chnames'][:,0] #[:,0] converts from 2D array back to 1D array
    print("Channel names: {channelNames}".format(channelNames=chnames))
    markers = o['marker']
    ## The markers are all still individual arrays of size 1x1, so we convert them to an array with single values
    markersArray = []
    for marker in markers:
        markersArray.append(marker[0])
    markersArray = np.asarray(markersArray)
    #For this dataset, the markers are 0, 1, or 2.
    # 1 - Left Hand MI, 2 - Right Hand MI, 3 - Passive State, 0 - Rest (???)

In [4]:
## Find the starting indeces where the marker changes
changeIdxs = np.where(np.transpose(markersArray)[:-1] != np.transpose(markersArray)[1:])[0]
print("Number of index changes: {idxChanges}".format(idxChanges=changeIdxs.shape[0]))
## Split the data so that it has its matching marker
dataSplit = np.array_split(data, changeIdxs[:-1], axis=1)
splitCount = 0
for splitData in dataSplit:
    splitCount += 1
print("Number of arrays in data split: {num}".format(num=splitCount))
## Retrieve the marker values for each of the change indeces (changeIdxs)
markerTargets = markersArray[changeIdxs];
print("Number of marker targets: {numTargets}".format(numTargets=markerTargets.shape[0]))

Number of index changes: 32336
Number of arrays in data split: 32336
Number of marker targets: 32336


In [5]:
## To Apply CSP, we first only get the indeces for MI tasks 1 and 2 (left and right hand, respectively.)
LeftIdxs = np.where(markerTargets == 1)
RightIdxs = np.where(markerTargets == 2)
numLeftIdx = LeftIdxs[0].shape
maxsteps=220 #most actions have lengths < 220

In [6]:
leftData = [];
for leftIndex in LeftIdxs[0]:
    #print(leftIndex)
    #print("Dimensions of index: {ind}".format(ind=dataSplit[leftIndex].shape))
    if(dataSplit[leftIndex].shape[1] > maxsteps):
        continue
    else:
        diff=maxsteps-dataSplit[leftIndex].shape[1]
        temp=np.pad(dataSplit[leftIndex], ((0,0),(0,diff)), mode='constant')
        leftData.append(np.transpose(temp))
leftData = np.asarray(leftData)
leftData.shape

(5490, 220, 21)

In [7]:
rightData = [];
for rightIndex in RightIdxs[0]:
    #print(leftIndex)
    #print("Dimensions of index: {ind}".format(ind=dataSplit[leftIndex].shape))
    if(dataSplit[rightIndex].shape[1] > maxsteps):
        continue
    else:
        diff=maxsteps-dataSplit[rightIndex].shape[1]
        temp=np.pad(dataSplit[rightIndex], ((0,0),(0,diff)), mode='constant')
        rightData.append(np.transpose(temp))
rightData = np.asarray(rightData)
rightData.shape

(5605, 220, 21)

In [8]:
## Only keep the top 5490 samples, so that left and right data are equal
rightDataSub = rightData[1:5491]

In [9]:
#Construct the target array and merge the data
leftTargets = np.tile(np.array([1,0]),(5490,1))
rightTargets = np.tile(np.array([0,1]), (5490,1))
markerTargets = np.vstack((leftTargets, rightTargets))
lrData = np.vstack((leftData, rightDataSub))

#Sanity Check
print("lrData Shape: {arg1}\tmarkerTargets Shape: {arg2}".format(arg1=lrData.shape, arg2=markerTargets.shape))

lrData Shape: (10980, 220, 21)	markerTargets Shape: (10980, 2)


In [10]:
## Construct LSTM using Tensorflow + Keras
# Import Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras import optimizers

In [11]:
## Shuffle the data
lrData, markerTargets = shuffle(lrData, markerTargets, random_state=0)

In [12]:
## Split into train and test sets
lrDataTrain, lrDataTest, markerTargetsTrain, markerTargetsTest = train_test_split(lrData, markerTargets, test_size=0.3, random_state=1)
markerTargetsTrain.shape

(7686, 2)

In [13]:
## Reshape the data for time-series processing
## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
lrDataTrainRe = lrDataTrain.reshape((lrDataTrain.shape[0], lrDataTrain.shape[1], lrDataTrain.shape[2]))
lrDataTestRe = lrDataTest.reshape((lrDataTest.shape[0], lrDataTest.shape[1], lrDataTest.shape[2]))

In [14]:
## Construct the model
LSTM_EEG = Sequential()
LSTM_EEG.add(LSTM((100),batch_input_shape=(None,lrDataTrainRe.shape[1], lrDataTrainRe.shape[2]), return_sequences=True))
LSTM_EEG.add(LSTM((50), return_sequences=False))
LSTM_EEG.add(Dense((2),activation='sigmoid'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
LSTM_EEG.summary()
sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
LSTM_EEG.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 220, 100)          48800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense (Dense)                (None, 2)                 102       
Total params: 79,102
Trainable params: 79,102
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
history = LSTM_EEG.fit(lrDataTrain, markerTargetsTrain, epochs=5,verbose=2, batch_size=256)

Epoch 1/5
7686/7686 - 26s - loss: 0.6938 - acc: 0.5083
Epoch 2/5
7686/7686 - 26s - loss: 0.6930 - acc: 0.5184
Epoch 3/5
7686/7686 - 27s - loss: 0.6903 - acc: 0.5387
Epoch 4/5
7686/7686 - 27s - loss: 0.6880 - acc: 0.5405
Epoch 5/5
7686/7686 - 26s - loss: 0.6851 - acc: 0.5443


In [17]:
predictionsTest = LSTM_EEG.predict(lrDataTest)

In [18]:
predictionsTest[predictionsTest>0.5] = 1

In [19]:
predictionsTest[predictionsTest <= 0.5] = 0

In [20]:
comparisonArrayTest = predictionsTest == markerTargetsTest

In [21]:
correctCountTest = 0
for boolValues in comparisonArrayTest:
    if(boolValues[0] & boolValues[1]):
        correctCountTest += 1
falseCountTest = lrDataTest.shape[0] - correctCountTest

predictionsTrain = LSTM_EEG.predict(lrDataTrain)
predictionsTrain[predictionsTrain>0.5] = 1;
predictionsTrain[predictionsTrain<=0.5] = 0;
comparisonArrayTrain = predictionsTrain == markerTargetsTrain;

correctCountTrain = 0
for boolValues in comparisonArrayTrain:
    if(boolValues[0] & boolValues[1]):
        correctCountTrain += 1
falseCountTrain = lrDataTrain.shape[0] - correctCountTrain

In [22]:
print("#################################")
print("#################################")
print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, (correctCountTrain*100/lrDataTrain.shape[0])))
print("#################################")
print("#################################")
print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, (correctCountTest*100/lrDataTest.shape[0])))
print("#################################")
print("#################################")

#################################
#################################
Training Performance:
Correct MI Prediction: 4189
Incorrect MI Prediction: 3497
Percent Accuracy: 54.502%
#################################
#################################
Testing Performance:
Correct MI Prediction: 1674
Incorrect MI Prediction: 1620
Percent Accuracy: 50.820%
#################################
#################################
